Copyright (c) 2020-2021 Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Demo of AutoML with FLAML Library


## 1. Introduction

FLAML is a Python library (https://github.com/microsoft/FLAML) designed to automatically produce accurate machine learning models 
with low computational cost. It is fast and cheap. The simple and lightweight design makes it easy 
to use and extend, such as adding new learners. FLAML can 
- serve as an economical AutoML engine,
- be used as a fast hyperparameter tuning tool, or 
- be embedded in self-tuning software that requires low latency & resource in repetitive
   tuning tasks.

In this notebook, we demonstrate how to ues FLAML library to tune hyperparameters of LightGBM with a regression example.

FLAML requires `Python>=3.6`. To run this notebook example, please install flaml with the `notebook` option:
```bash
pip install flaml[notebook]
```

In [1]:
!pip install flaml[notebook];

## 2. Real Data Example
### Load data and preprocess

Download [Houses dataset](https://www.openml.org/d/537) from OpenML. The task is to predict median price of the house in the region based on demographic composition and a state of housing market in the region.

In [22]:
from flaml.data import load_openml_dataset
X_train, X_test, y_train, y_test = load_openml_dataset(dataset_id = 537, data_dir = './')

download dataset from openml
Dataset name: houses
X_train.shape: (15480, 8), y_train.shape: (15480,);
X_test.shape: (5160, 8), y_test.shape: (5160,)


### Run FLAML
In the FLAML automl run configuration, users can specify the task type, time budget, error metric, learner list, whether to subsample, resampling strategy type, and so on. All these arguments have default values which will be used if users do not provide them. 

In [2]:
''' import AutoML class from flaml package '''
from flaml import AutoML
automl = AutoML()

In [40]:
settings = {
    "time_budget": 180, # total running time in seconds
    "metric": 'r2', # primary metrics for regression can be chosen from: ['mae','mse','r2']
    "estimator_list": ['lgbm'], # list of ML learners; we tune lightgbm in this example
    "task": 'regression', # task type    
    "log_file_name": 'houses_experiment.log', # flaml log file
}

In [41]:
'''The main flaml automl API'''
automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 02-22 11:53:25] {839} INFO - Evaluation method: cv
[flaml.automl: 02-22 11:53:25] {568} INFO - Using RepeatedKFold
[flaml.automl: 02-22 11:53:25] {860} INFO - Minimizing error metric: 1-r2
[flaml.automl: 02-22 11:53:25] {880} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 02-22 11:53:25] {939} INFO - iteration 0  current learner lgbm
[flaml.automl: 02-22 11:53:25] {1093} INFO -  at 0.2s,	best lgbm's error=0.7383,	best lgbm's error=0.7383
[flaml.automl: 02-22 11:53:25] {939} INFO - iteration 1  current learner lgbm
[flaml.automl: 02-22 11:53:25] {1093} INFO -  at 0.3s,	best lgbm's error=0.7383,	best lgbm's error=0.7383
[flaml.automl: 02-22 11:53:25] {939} INFO - iteration 2  current learner lgbm
[flaml.automl: 02-22 11:53:25] {1093} INFO -  at 0.4s,	best lgbm's error=0.4578,	best lgbm's error=0.4578
[flaml.automl: 02-22 11:53:25] {939} INFO - iteration 3  current learner lgbm
[flaml.automl: 02-22 11:53:25] {1093} INFO -  at 0.5s,	best lgbm's error=0.457

### Best model and metric

In [42]:
''' retrieve best config'''
print('Best hyperparmeter config:', automl.best_config)
print('Best r2 on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best hyperparmeter config: {'n_estimators': 451.0, 'max_leaves': 113.0, 'min_child_weight': 20.0, 'learning_rate': 0.025065630491840726, 'subsample': 1.0, 'log_max_bin': 8.0, 'colsample_bytree': 0.9046814915274195, 'reg_alpha': 8.352751749829367e-10, 'reg_lambda': 0.13991138691596908}
Best r2 on validation data: 0.8437
Training duration of best run: 15.14 s


In [7]:
automl.model

LGBMRegressor(learning_rate=0.02944211100744363, max_bin=1023,
              min_child_weight=20.0, n_estimators=329, num_leaves=17,
              objective='regression', reg_alpha=3.617531807484476e-05,
              reg_lambda=1.0)

In [8]:
''' pickle and save the best model '''
import pickle
with open('best_model.pkl', 'wb') as f:
    pickle.dump(automl.model, f, pickle.HIGHEST_PROTOCOL)

In [43]:
''' compute predictions of testing dataset ''' 
y_pred = automl.predict(X_test)
print('Predicted labels', y_pred)
print('True labels', y_test)

Predicted labels [147056.672508   246591.18821626 155253.69332074 ... 196516.76693923
 235571.37776252 270133.77185961]
True labels [136900. 241300. 200700. ... 160900. 227300. 265600.]


In [44]:
''' compute different metric values on testing dataset'''
from flaml.ml import sklearn_metric_loss_score
print('r2', '=', 1 - sklearn_metric_loss_score('r2', y_pred, y_test))
print('mse', '=', sklearn_metric_loss_score('mse', y_pred, y_test))
print('mae', '=', sklearn_metric_loss_score('mae', y_pred, y_test))

r2 = 0.8503723727607084
mse = 1977853769.4384706
mae = 29258.487121555943


In [11]:
from flaml.data import get_output_from_log
time_history, best_valid_loss_history, valid_loss_history, config_history, train_loss_history = \
    get_output_from_log(filename = settings['log_file_name'], time_budget = 60)

for config in config_history:
    print(config)

{'Current Learner': 'lgbm', 'Current Sample': 10000, 'Current Hyper-parameters': {'n_estimators': 4, 'max_leaves': 4, 'min_child_weight': 20.0, 'learning_rate': 0.1, 'subsample': 1.0, 'log_max_bin': 8, 'colsample_bytree': 1.0, 'reg_alpha': 1e-10, 'reg_lambda': 1.0, 'FLAML_sample_size': 10000}, 'Best Learner': 'lgbm', 'Best Hyper-parameters': {'n_estimators': 4, 'max_leaves': 4, 'min_child_weight': 20.0, 'learning_rate': 0.1, 'subsample': 1.0, 'log_max_bin': 8, 'colsample_bytree': 1.0, 'reg_alpha': 1e-10, 'reg_lambda': 1.0, 'FLAML_sample_size': 10000}}
{'Current Learner': 'lgbm', 'Current Sample': 10000, 'Current Hyper-parameters': {'n_estimators': 4.0, 'max_leaves': 4.0, 'min_child_weight': 20.0, 'learning_rate': 0.46335414315327306, 'subsample': 0.9339389930838808, 'log_max_bin': 10.0, 'colsample_bytree': 0.9904286645657556, 'reg_alpha': 2.841147337412889e-10, 'reg_lambda': 0.12000833497054482, 'FLAML_sample_size': 10000}, 'Best Learner': 'lgbm', 'Best Hyper-parameters': {'n_estimator

In [29]:
import matplotlib.pyplot as plt
import numpy as np

plt.title('Learning Curve')
plt.xlabel('Wall Clock Time (s)')
plt.ylabel('Validation r2')
plt.scatter(time_history, 1-np.array(valid_loss_history))
plt.step(time_history, 1-np.array(best_valid_loss_history), where='post')
plt.show()

NameError: name 'time_history' is not defined

## 3. Comparison with alternatives

### FLAML's accuracy

In [31]:
print('flaml r2', '=', 1 - sklearn_metric_loss_score('r2', y_pred, y_test))

flaml r2 = 0.8503723727607084


In [35]:
settings = {
    "time_budget": 300, # total running time in seconds
    "metric": 'r2', 
    "task": 'regression', # task type    
    "estimator_list": ['lgbm'],
    "log_file_name": 'houses_experiment_optuna.log', 
    "hpo_method": 'optuna',
}
automl.fit(X_train = X_train, y_train = y_train, **settings)
y_pred = automl.predict(X_test)
print('flaml r2 using optuna as the hpo method', '=', 1 - sklearn_metric_loss_score('r2', y_pred, y_test))

[flaml.automl: 02-22 10:47:51] {839} INFO - Evaluation method: cv
[flaml.automl: 02-22 10:47:51] {568} INFO - Using RepeatedKFold
[flaml.automl: 02-22 10:47:51] {860} INFO - Minimizing error metric: 1-r2
[flaml.automl: 02-22 10:47:51] {880} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 02-22 10:47:51] {939} INFO - iteration 0  current learner lgbm
[flaml.searcher.suggestion: 02-22 10:47:51] {518} WARNING - You passed a `space` parameter to <class 'flaml.searcher.suggestion.OptunaSearch'> that contained unresolved search space definitions. <class 'flaml.searcher.suggestion.OptunaSearch'> should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[flaml.searcher.suggestion: 02-22 10:47:51] {655} WARNING - Optuna does not support both quantization and sampling from LogUniform. Dropped quantization.
[flaml.searcher.sugges

### Default LightGBM

In [32]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor()

In [33]:
lgbm.fit(X_train, y_train)

LGBMRegressor()

In [34]:
y_pred = lgbm.predict(X_test)
from flaml.ml import sklearn_metric_loss_score
print('default lgbm r2', '=', 1 - sklearn_metric_loss_score('r2', y_pred, y_test))

default lgbm r2 = 0.8296179648694404


### Optuna LightGBM Tuner

In [37]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(X_train, y_train, test_size=0.1)
import optuna.integration.lightgbm as lgb
dtrain = lgb.Dataset(train_x, label=train_y)
dval = lgb.Dataset(val_x, label=val_y)
params = {
    "objective": "regression",
    "metric": "regression",
    "verbosity": -1,
}


In [38]:
%%time
model = lgb.train(params, dtrain, valid_sets=[dtrain, dval], verbose_eval=10000)        


6 and parameters: {'feature_fraction': 0.8999999999999999}. Best is trial 0 with value: 2183189353.109814.
bagging, val_score: 2120154666.907345: 100%|##########| 10/10 [00:18<00:00,  1.87s/it]
feature_fraction_stage2, val_score: 2120154666.907345:  17%|#6        | 1/6 [00:02<00:11,  2.36s/it][I 2021-02-22 10:58:20,858] Trial 37 finished with value: 2120154666.9073446 and parameters: {'feature_fraction': 0.6799999999999999}. Best is trial 37 with value: 2120154666.9073446.
feature_fraction_stage2, val_score: 2120154666.907345:  33%|###3      | 2/6 [00:04<00:09,  2.29s/it][I 2021-02-22 10:58:23,103] Trial 38 finished with value: 2120154666.9073446 and parameters: {'feature_fraction': 0.616}. Best is trial 37 with value: 2120154666.9073446.
feature_fraction_stage2, val_score: 2120154666.907345:  50%|#####     | 3/6 [00:06<00:06,  2.24s/it][I 2021-02-22 10:58:25,279] Trial 39 finished with value: 2340044642.568378 and parameters: {'feature_fraction': 0.52}. Best is trial 37 with value: 21

In [39]:
y_pred = model.predict(X_test)
from flaml.ml import sklearn_metric_loss_score
print('Optuna LightGBM Tuner r2', '=', 1 - sklearn_metric_loss_score('r2', y_pred, y_test))

Optuna LightGBM Tuner r2 = 0.8451504635149211
